# Introducción al Aprendizaje Automático

## Aprendizaje Supervisado - Métricas para Clasificación y Evaluación de Modelos

#### Conjunto de datos

**Ejercicio:** Cargar el mismo dataset de la guía anterior. En una variable `X` seleccionar los atributos y en una variable `y` el target.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Lic_en_Datos/IAA/datos/IAA_Guia_2_clasificacion_binaria.csv')  #drive mica
data.sample(5)

,x1,x2,target
287,-3.087621,-0.986066,0.0
181,-1.584899,-1.663652,1.0
179,-3.467293,0.520245,0.0
394,0.345923,0.542885,1.0
261,-0.767029,-1.323192,1.0


In [ ]:
  data.shape

(450, 3)

**Ejercicio:** Separar `X` e `y` en datos de entrenamiento (`X_train,y_train`) y de evaluación (`X_test,y_test`), usando la función `train_test_split` de Scikit-Learn. Toma un 70% para entrenamiento y un 30% para evaluación. Recuerda mirar su documentación y fijar el `random_state`. Por último, ten en cuenta que con `X_test,y_test` **no vamos a entrenar el modelo, sino que vamos a usarlos únicamente para evaluar su desempeño**.


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = data[['x1', 'x2']]
y = data["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) #test_size = 0.3 da 70% entrenamiento y 30% evaluacion

In [ ]:
print("Shape de X:", X.shape)
print("Shape de y:", y.shape)

Shape de X: (450, 2)
Shape de y: (450,)


**Ejercicio:** Entrena un árbol de decisión de profundidad máxima 3.


**Ejercicio:** visualiza la frontera de decisión usando la función `visualize_classifier` de la guía anterior. Grafica los puntos de entrenamiento y evaluación, con sus respectivas clases (puedes usar el parámetro alpha, otro marker o cambiar el tamaño para diferenciar entre entrenamiento y evaluación).

In [ ]:
# Función que nos ayuda a graficar
# No hace falta que comprandan este bloque de código.
def visualize_classifier(model, X, y, ax=None, proba = False):
    if isinstance(X, pd.DataFrame):
        X = X.values
    if isinstance(y, pd.Series):
        y = y.values
    ax = ax or plt.gca()

    colors_tab10 = plt.cm.tab10.colors

    for i, y_value in enumerate(reversed(np.unique(y))):
        ax.scatter(X[y==y_value, 0], X[y==y_value, 1], s=30,
                   zorder=3, alpha = 0.5, color = colors_tab10[i])

    ax.axis('tight')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    # ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                         np.linspace(*ylim, num=200))

    if proba:
        Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1].reshape(xx.shape)
    else:
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)


    # Create a color plot with the results
    Z = -Z + 1
    ax.pcolormesh(xx,yy,Z,cmap='bwr', vmin = 0, vmax=1, alpha = 0.2)

    ax.set(xlim=xlim, ylim=ylim)


In [ ]:
# COMPLETAR

**Ejercicio:** elije una clase como la positiva y otra clase como la negativa. Escribe una función que calcula la matriz de confusión. Aplícala tanto para entrenamiento como para evaluación. No es necesario presentarla en forma de matriz, pero sí es necesario que calcules los totales de verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos.

In [ ]:
def confusion_matrix(y, y_pred, positive_class = 1):
    # COMPLETAR
    pass

In [ ]:
y_train_pred = clf.predict(# COMPLETAR)
y_test_pred = # COMPLETAR

In [ ]:
print(confusion_matrix(# COMPLETAR))
print(confusion_matrix(# COMPLETAR))

**Ejercicio:** A partir de las matrices de confusión obtenidas, escribe funciones que calculen la exactitud, la precisión, la exhaustividad (recall) y el F1-score para el clasificador. Es conveniente que llamen a la función que escribiste antes. Aplicar tanto sobre entrenamiento como sobre evaluación.

In [ ]:
def exactitud(y, y_pred):
    # COMPLETAR
    pass

def precision(y, y_pred):
    # COMPLETAR
    pass

def recall(y, y_pred):
    # COMPLETAR
    pass

def f_score(y, y_pred):
    # COMPLETAR
    pass

**Ejercicio:** haz un gráfico de exactitud - calculada en entrenamiento y evaluación - en función de `max_depth` para el árbol de decisión. ¿Qué observas? ¿Qué profundidad elegirías para un árbol de decisión en este caso?¿Por qué?

In [ ]:
max_depths = np.arange(1, 21)
train_scores = []
test_scores = []
for max_depth in max_depths:
    clf = # COMPLETAR
    # COMPLETAR

    y_train_pred = # COMPLETAR
    # COMPLETAR

    train_scores.append(# COMPLETAR)
    test_scores.append(# COMPLETAR)

In [ ]:
# COMPLETAR

**Ejercicio:** buscar en la documentación de Scikit-Learn cómo se calcula la matriz de confusión, las métricas mencionadas y el `classification_report`. Aplicarlos.

## 2. Conjunto de datos de Pingüinos

Aplicar los visto al conjunto de datos de Pingüinos. Para ello:

1. Evaluar un clasificador para predecir el género de un pingüino a partir de sus atributos físicos. Hacerlo por medio de train-test split.

1. Evaluar un clasificador para predecir la especie de un pingüino a partir de sus atributos físicos. Hacerlo por medio de train-test split.

Utilizar las herramientas provistas por Scikit-Learn.

In [ ]:
# COMPLETAR


## 3. Más ejercicios

**Ejercicio - Métricas:**
1. En un problema de clasificación binaria, ¿a qué se denomina clase positiva y a qué clase negativa? Si nuestro problema consiste en clasificar spam vs. no-spam, ¿cuál es la clase positiva? Si nuestro problema es clasificar imágenes de perros vs. gatos, ¿cuál es la clase positiva?

1. Explicar con tus palabras la definición de verdadero positivo, verdadero negativo, falso positivo y falso negativo.

1. ¿Por qué podría un falso positivo ser considerado más (o menos) importante que un falso negativo? Dar un ejemplo en donde es más grave tener falsos negativos que falsos positivos.

1. Explicar con tus palabras la definición de accuracy, precision y recall. ¿En qué situaciones serán más útiles cada una de estas métricas?

1. ¿Por qué es un problema medir accuracy de un clasificador para compararlo con otro? (Pensar en desbalance de clases.) Dar un ejemplo en donde sería engañoso utilizar esta comparación.

1. Construyes un clasificador que devuelve siempre la clase positiva. ¿Qué exactitud, precisión y recall esperarías para este clasificador?¿Hace falta más información?

**Ejercicio:** Un modelo tiene una exactitud de 0.90 en entrenamiento y 0.60 en evaluación. ¿Qué puede decir sobre el modelo?

**Ejercicio:** La exhaustividad/recall/TPR y la precisión no tienen en cuenta los verdaderos negativos. ¿Existe alguna métrica que sí lo haga?¿Por qué podría ser útil? Existe una que es "hermana" de la exhaustividad. Averigua cómo se llama y en qué contextos suele aplicarse.

**Ejercicio:** Supongamos que un clasificador devuelve etiquetas al azar en un contexto binario, balanceado. ¿Qué valores esperarías para la exactitud, la precisión y la exhaustividad?¿Por qué?

**Ejercicio:** Repite el ejercicio anterior, pero ahora en lugar de dos clases, tienes tres. Solamente hazlo para exactitud. ¿Y con cuatro?¿Y con *n* clases?


**Ejercicio:** Explicar que es el subajuste y el sobreajuste. Elegir algún problema binario visto o inventado durante las clases anteriores y dibujar la frontera de decisión para el caso con dos atributos de un modelo que esté subajustado y otro que esté sobreajustado.


**Ejercicio:** Verdadero o Falso. Justificar.

1. Un árbol de decisión puede conseguir 100 % de aciertos en los datos de entrenamiento.
1. Un árbol de decisión consigue 100 % de aciertos en los datos de entrenamiento siempre y cuando no haya contradicciones entre las etiquetas de instancias muy similares.


**Ejercicio:** en la clase teórica vimos que al hacer train-test split  los datos no siempre deben separarse al azar. ¿Por qué? Pensar al menos dos situaciones en las cuales no sea conveniente.

